In [1]:
# Code for solving the Roeder problem:
#
#  Find the longest sequence using numbers from 
#  {1,...,100} such that every number is either 
#  a factor or multiple of the previous number. 
#
# https://blog.computationalcomplexity.org/2022/04/the-roeder-problem-was-solved-before-i.html

In [2]:
import networkx as nx

# creates divisor graph on nodes { 1, 2, ..., n } 
#   with two special nodes named 0 and n+1 that
#   are to be the start and end of our path.

def create_divisor_graph(n):
    
    G = nx.Graph()
    G.add_nodes_from(range(n+2))
    
    # add edges from 0 to all other nodes
    for i in range(1,n+1):
        G.add_edge(0,i)
    
    # add edges from all nodes to node n+1
    for i in range(1,n+1):
        G.add_edge(i,n+1)
    
    # add edges between divisors
    for i in range(1,n):
        for m in range(2,n):
            v = i * m
            if v > n:
                break
            G.add_edge(i,v)
            
    return G

In [3]:
import gurobipy as gp
from gurobipy import GRB

In [4]:
# create a function to separate the subtour elimination constraints
def subtour_elimination(m, where):
    
    # check if LP relaxation at this branch-and-bound node has an integer solution
    if where == GRB.Callback.MIPSOL: 
        
        # retrieve the LP solution
        xval = m.cbGetSolution(m._x)
        
        # which edges are selected?
        edges = [ e for e in m._G.edges if xval[e] > 0.5 ]
        
        # for each subtour, add a constraint
        for component in nx.connected_components( m._G.edge_subgraph( edges ) ):
            
            if m._s not in component:
            
                inner_edges = [ (i,j) for (i,j) in m._G.edges if i in component and j in component ]
                m.cbLazy( gp.quicksum( m._x[e] for e in inner_edges ) <= len(component) - 1 )    

In [5]:
def solve_Roeder_problem(n):

    G = create_divisor_graph(n)
    
    s = 0
    t = n+1

    m = gp.Model()
    x = m.addVars( G.edges, vtype=GRB.BINARY )
    y = m.addVars( G.nodes, vtype=GRB.BINARY )

    y[s].ub = 0
    y[t].ub = 0

    # maximize number of nodes in the s,t-path (excludes s and t)
    m.setObjective( gp.quicksum( y[i] for i in G.nodes ), GRB.MAXIMIZE )

    # one edge should touch node s (and node t)
    m.addConstr( gp.quicksum( x[e] for e in G.edges if e in G.edges(s) ) == 1 )
    m.addConstr( gp.quicksum( x[e] for e in G.edges if e in G.edges(t) ) == 1 )

    # all other nodes should have degree 2
    m.addConstrs( gp.quicksum( x[e] for e in G.edges if e in G.edges(i) ) == 2 * y[i] for i in G.nodes if i not in {s,t} )

    # tell Gurobi that we will be adding (lazy) constraints
    m.Params.lazyConstraints = 1

    # designate the callback routine to be subtour_elimination()
    m._callback = subtour_elimination

    # add the variables and graph to our model object, for use in the callback
    m._x = x
    m._G = G
    m._s = s

    # solve the MIP with our callback
    m.Params.TimeLimit = 2 * 3600      # two hours
    m.optimize(m._callback)
    
    edges = [ (i,j) for (i,j) in G.edges if x[i,j].x > 0.5 ]
    path = nx.shortest_path(G.edge_subgraph(edges), source=s, target=t)
    print(path[1:-1])  # print the path, except for s and t
    return     

In [6]:
solve_Roeder_problem(n=10)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-01
Set parameter LazyConstraints to value 1
Set parameter TimeLimit to value 7200
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 12 rows, 48 columns and 82 nonzeros
Model fingerprint: 0x1635d53b
Variable types: 0 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1.0000000
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s
Presolved: 12 rows, 46 columns, 82 nonzeros
Variable types: 0 continuous, 46 integer (46 binary)

Root relaxation: objective 9.000000e+00, 22 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf 

In [7]:
solve_Roeder_problem(n=100)

Set parameter LazyConstraints to value 1
Set parameter TimeLimit to value 7200
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 102 rows, 683 columns and 1262 nonzeros
Model fingerprint: 0xf3b5c884
Variable types: 0 continuous, 683 integer (683 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s
Presolved: 102 rows, 681 columns, 1262 nonzeros
Variable types: 0 continuous, 681 integer (681 binary)
Found heuristic solution: objective 1.0000000

Root relaxation: objective 8.150000e+01, 184 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   81.50000    0   14 

In [8]:
solve_Roeder_problem(n=200)

Set parameter LazyConstraints to value 1
Set parameter TimeLimit to value 7200
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 202 rows, 1499 columns and 2794 nonzeros
Model fingerprint: 0xbc942751
Variable types: 0 continuous, 1499 integer (1499 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1.0000000
Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 202 rows, 1497 columns, 2794 nonzeros
Variable types: 0 continuous, 1497 integer (1497 binary)

Root relaxation: objective 1.625000e+02, 403 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  162.50000    0

In [9]:
solve_Roeder_problem(n=300)

Set parameter LazyConstraints to value 1
Set parameter TimeLimit to value 7200
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 302 rows, 2368 columns and 4432 nonzeros
Model fingerprint: 0xd795dff0
Variable types: 0 continuous, 2368 integer (2368 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s
Presolved: 302 rows, 2366 columns, 4432 nonzeros
Variable types: 0 continuous, 2366 integer (2366 binary)
Found heuristic solution: objective 1.0000000

Root relaxation: objective 2.425000e+02, 567 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  242.50000    0

In [10]:
solve_Roeder_problem(n=400)

Set parameter LazyConstraints to value 1
Set parameter TimeLimit to value 7200
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 402 rows, 3269 columns and 6134 nonzeros
Model fingerprint: 0x4a5c40e8
Variable types: 0 continuous, 3269 integer (3269 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 402 rows, 3267 columns, 6134 nonzeros
Variable types: 0 continuous, 3267 integer (3267 binary)
Found heuristic solution: objective 1.0000000

Root relaxation: objective 3.225000e+02, 752 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  322.50000    0

In [11]:
solve_Roeder_problem(n=500)

Set parameter LazyConstraints to value 1
Set parameter TimeLimit to value 7200
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 502 rows, 4191 columns and 7878 nonzeros
Model fingerprint: 0x85715bd5
Variable types: 0 continuous, 4191 integer (4191 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 502 rows, 4189 columns, 7878 nonzeros
Variable types: 0 continuous, 4189 integer (4189 binary)
Found heuristic solution: objective 1.0000000

Root relaxation: objective 4.030000e+02, 901 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  403.00000    0

In [12]:
solve_Roeder_problem(n=600)

Set parameter LazyConstraints to value 1
Set parameter TimeLimit to value 7200
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 602 rows, 5145 columns and 9686 nonzeros
Model fingerprint: 0x83ce66d8
Variable types: 0 continuous, 5145 integer (5145 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.02s
Presolved: 602 rows, 5143 columns, 9686 nonzeros
Variable types: 0 continuous, 5143 integer (5143 binary)
Found heuristic solution: objective 1.0000000

Root relaxation: objective 4.820000e+02, 1108 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  482.00000    

In [13]:
solve_Roeder_problem(n=1000)

Set parameter LazyConstraints to value 1
Set parameter TimeLimit to value 7200
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1002 rows, 9070 columns and 17136 nonzeros
Model fingerprint: 0x83eeb01c
Variable types: 0 continuous, 9070 integer (9070 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 1002 rows, 9068 columns, 17136 nonzeros
Variable types: 0 continuous, 9068 integer (9068 binary)
Found heuristic solution: objective 1.0000000

Root relaxation: objective 8.025000e+02, 1805 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  802.50000

 75637 52017  679.70669   39  219  653.00000  684.50415  4.82%  33.6  345s
 76600 52769  683.51752   36  310  653.00000  684.49357  4.82%  33.7  350s
 78160 53875  680.92967   48  232  653.00000  684.49035  4.82%  33.7  357s
 79013 54660  678.73389   32  238  653.00000  684.48418  4.82%  33.8  361s
 80289 55491  655.78941   84  117  653.00000  684.47226  4.82%  33.8  365s
 81323 56275  671.57625   59  218  653.00000  684.46226  4.82%  33.8  370s
 83059 57458  675.11054   61  196  653.00000  684.44635  4.82%  33.9  377s
 84183 58208     cutoff   88       653.00000  684.43845  4.81%  33.9  382s
 85296 59022  671.42157   69  188  653.00000  684.42817  4.81%  34.0  387s
 85751 59051  675.44075   50  227  653.00000  684.42327  4.81%  34.1  392s
 85780 59568  672.73026   58  227  653.00000  684.42316  4.81%  34.1  396s
 87073 60393     cutoff   82       653.00000  684.41777  4.81%  34.1  402s
 88081 61264  672.97774   63  189  653.00000  684.41607  4.81%  34.1  407s
 88716 61687  678.79149  

 179512 124702  682.61618   33  223  655.00000  683.90839  4.41%  36.3  892s
 180172 125157  681.77599   31  261  655.00000  683.90102  4.41%  36.3  895s
 181453 125999 infeasible   60       655.00000  683.89783  4.41%  36.3  900s
 182615 127043 infeasible   69       655.00000  683.89468  4.41%  36.2  906s
 184115 128111  677.14451   41  179  655.00000  683.88834  4.41%  36.2  911s
 185091 128766  680.96626   32  246  655.00000  683.88252  4.41%  36.2  916s
 186421 129755  656.75000  148   51  655.00000  683.87400  4.41%  36.2  922s
 187589 130641  657.40278  156   80  655.00000  683.87237  4.41%  36.3  927s
 188295 131159  681.18435   41  239  655.00000  683.87136  4.41%  36.3  930s
 189526 131984  668.26471   76  139  655.00000  683.86633  4.41%  36.2  935s
 190900 132807  679.79149   53  207  655.00000  683.86097  4.41%  36.2  941s
 191941 133539  676.64443   51  164  655.00000  683.85474  4.41%  36.3  947s
 192481 134015  664.09704   86  158  655.00000  683.85351  4.41%  36.3  950s

 288344 203451  674.73554   68  181  655.00000  683.53586  4.36%  36.9 1452s
 288934 203943  680.47304   48  168  655.00000  683.53549  4.36%  36.9 1456s
 290227 204816  680.61054   44  192  655.00000  683.53240  4.36%  36.9 1462s
 290713 205173  677.61054   52  177  655.00000  683.53236  4.36%  36.9 1465s
 291896 205963  676.92554   64  145  655.00000  683.52826  4.36%  36.9 1471s
 293065 206668     cutoff   98       655.00000  683.52477  4.35%  37.0 1476s
 293542 207112  672.76025   55  206  655.00000  683.52222  4.35%  37.0 1480s
 294813 207949  680.30791   39  154  655.00000  683.51930  4.35%  37.0 1486s
 295867 208891  675.27654   41  217  655.00000  683.51772  4.35%  37.0 1493s
 296531 209353  682.01660   44  244  655.00000  683.51653  4.35%  37.0 1496s
 297568 210039  671.93111   61  233  655.00000  683.51371  4.35%  37.0 1501s
 298840 210924  677.73707   40  207  655.00000  683.51056  4.35%  37.0 1507s
 299903 211780  674.19387   60  198  655.00000  683.50862  4.35%  37.0 1512s

 392878 253509  667.85191   74  130  659.00000  683.26025  3.68%  37.6 1980s
 394010 254303  674.31887   52  163  659.00000  683.25808  3.68%  37.6 1985s
 395240 255157  670.34804   69  168  659.00000  683.25416  3.68%  37.6 1991s
 396501 255894  669.02500   76   92  659.00000  683.24895  3.68%  37.6 1997s
 397071 256148  681.77721   46  147  659.00000  683.24713  3.68%  37.7 2000s
 398187 256844  681.32943   49  269  659.00000  683.24672  3.68%  37.7 2006s
 399475 257654  680.90978   54  258  659.00000  683.24387  3.68%  37.7 2012s
 400118 258011  680.36680   46  185  659.00000  683.24220  3.68%  37.7 2015s
 401230 258899  662.83159   67  233  659.00000  683.23803  3.68%  37.7 2021s
 402546 259739  677.15221   46  201  659.00000  683.23387  3.68%  37.7 2027s
 403183 260080 infeasible   70       659.00000  683.23167  3.68%  37.7 2030s
 404362 260906  674.70387   47  169  659.00000  683.22857  3.68%  37.7 2036s
 405769 261766  681.65896   42  225  659.00000  683.22160  3.68%  37.7 2042s

...

 870728 490355  676.47959   39  208  665.00000  682.42500  2.62%  38.9 4665s
 871908 491039  680.24223   47  272  665.00000  682.42254  2.62%  38.9 4672s
 872497 491470  669.85191   65  115  665.00000  682.42116  2.62%  38.9 4675s
 873628 492166  672.21457   56  192  665.00000  682.41960  2.62%  38.9 4682s
 874223 492548  671.86887   66  185  665.00000  682.41805  2.62%  38.9 4685s
 875418 493275  673.54636   57  224  665.00000  682.41532  2.62%  38.9 4692s
 876053 493655  669.75963   72  187  665.00000  682.41462  2.62%  38.9 4695s
 877175 494383 infeasible   45       665.00000  682.41301  2.62%  38.9 4702s
 877745 494774  679.62623   45  149  665.00000  682.41080  2.62%  38.9 4705s
 878970 495509  675.16054   54  161  665.00000  682.41054  2.62%  38.9 4712s
 879605 495974  672.41054   58  165  665.00000  682.40970  2.62%  38.9 4717s
 880299 496359  671.47423   53  223  665.00000  682.40892  2.62%  38.9 4720s
 881616 497148  681.29195   36  209  665.00000  682.40692  2.62%  38.9 4727s

 968834 551286  672.78554   58  136  665.00000  682.29387  2.60%  39.0 5201s
 970248 552128  677.32721   38  168  665.00000  682.29264  2.60%  39.0 5209s
 970869 552600  678.25704   51  170  665.00000  682.29185  2.60%  39.0 5214s
 971592 553045  681.04149   49  223  665.00000  682.29149  2.60%  39.0 5219s
 972266 553416  680.01858   58  163  665.00000  682.29130  2.60%  39.0 5224s
 972924 553785  673.53554   57  189  665.00000  682.29062  2.60%  39.0 5228s
 973530 554107  678.33396   49  159  665.00000  682.28993  2.60%  39.0 5231s
 974492 554831  679.67253   41  239  665.00000  682.28870  2.60%  39.0 5237s
 975104 555136 infeasible   62       665.00000  682.28850  2.60%  39.0 5241s
 976356 555988  669.85054   56  187  665.00000  682.28640  2.60%  39.0 5247s
 976982 556381  674.24387   39  168  665.00000  682.28582  2.60%  39.0 5251s
 977804 556913  680.54943   38  200  665.00000  682.28554  2.60%  39.0 5257s
 978452 557276  673.24982   60  172  665.00000  682.28482  2.60%  39.0 5260s

 1066128 590980  669.03554   64  146  666.00000  682.18882  2.43%  39.0 5726s
 1067292 591570  679.56025   47  223  666.00000  682.18799  2.43%  39.0 5732s
 1067814 591978  669.32721   59  177  666.00000  682.18716  2.43%  39.0 5735s
 1069067 592840  678.57721   47  175  666.00000  682.18630  2.43%  39.0 5741s
 1069685 593206  680.69146   43  210  666.00000  682.18552  2.43%  39.0 5745s
 1070956 593956  677.03554   48  199  666.00000  682.18495  2.43%  39.0 5751s
 1072113 594653  676.85054   56  180  666.00000  682.18323  2.43%  39.0 5756s
 1072663 595063  672.44863   71  221  666.00000  682.18265  2.43%  39.0 5760s
 1073900 595721  676.74387   50  218  666.00000  682.18054  2.43%  39.0 5765s
 1075032 596468  681.66999   45  155  666.00000  682.17888  2.43%  39.0 5771s
 1076392 597231  679.78659   42  246  666.00000  682.17786  2.43%  39.0 5777s
 1076985 597583  678.26054   54  211  666.00000  682.17721  2.43%  39.0 5781s
 1078169 598320  676.85054   59  149  666.00000  682.17547  2.43

 1166863 650551  679.13396   50  189  666.00000  682.09149  2.42%  39.1 6255s
 1167816 651264  669.38672   56  174  666.00000  682.09023  2.42%  39.1 6261s
 1168319 651597  679.52766   49  246  666.00000  682.09023  2.42%  39.1 6265s
 1169467 652308  679.96054   37  175  666.00000  682.08859  2.42%  39.1 6271s
 1170125 652719  676.85981   70  181  666.00000  682.08803  2.42%  39.1 6275s
 1171225 653388 infeasible   58       666.00000  682.08709  2.42%  39.1 6282s
 1171832 653771  674.40296   52  190  666.00000  682.08668  2.42%  39.1 6285s
 1173195 654555 infeasible   70       666.00000  682.08530  2.42%  39.1 6292s
 1173683 654882  681.78853   49  249  666.00000  682.08459  2.42%  39.1 6295s
 1174739 655525 infeasible   67       666.00000  682.08378  2.41%  39.1 6301s
 1175826 656086  677.92496   42  232  666.00000  682.08260  2.41%  39.1 6307s
 1176363 656429  669.35846   61  196  666.00000  682.08223  2.41%  39.1 6311s
 1177618 657267  670.66695   63  219  666.00000  682.08089  2.41

 1263018 706624  675.85191   58  166  666.00000  682.01054  2.40%  39.1 6796s
 1264306 707273  679.31666   44  207  666.00000  682.01011  2.40%  39.1 6802s
 1264866 707657  670.53554   62  133  666.00000  682.00943  2.40%  39.1 6805s
 1266208 708530  669.20221   71  162  666.00000  682.00862  2.40%  39.1 6812s
 1266919 708874  670.61887   58  156  666.00000  682.00837  2.40%  39.1 6815s
 1268218 709572  668.43236   70  201  666.00000  682.00740  2.40%  39.1 6821s
 1269406 710206  674.68525   58  193  666.00000  682.00624  2.40%  39.1 6827s
 1270004 710633  680.81588   36  266  666.00000  682.00570  2.40%  39.1 6831s
 1271277 711403  679.82721   53  212  666.00000  682.00404  2.40%  39.1 6837s
 1271938 711829  668.86337   55  216  666.00000  682.00340  2.40%  39.1 6840s
 1272945 712494  679.12487   45  213  666.00000  682.00237  2.40%  39.1 6846s
 1274060 713244  680.31287   37  233  666.00000  682.00152  2.40%  39.1 6853s
 1274657 713650  680.16978   36  238  666.00000  682.00118  2.40